# Reappropriation Comparison: 25-26 vs 26-27

This notebook extracts reappropriations from budget PDFs and compares them to identify spending authority that was dropped.

## Setup
1. Upload the 6 PDF files when prompted
2. Run all cells in order
3. Download the comparison Excel file at the end

In [ ]:
# Install required packages
!pip install pdfplumber pandas openpyxl -q
print("Packages installed successfully!")

In [ ]:
import pdfplumber
import pandas as pd
import re
from google.colab import files
import os

print("Libraries imported successfully!")

In [ ]:
# Upload PDF files
print("Please upload all 6 PDF files:")
print("  - ATL Reapprops 25-26.pdf")
print("  - CAP Reapprops 25-26.pdf")
print("  - STOPS Reapprops 25-26.pdf")
print("  - ATL 26-27 Reapprops.pdf")
print("  - CAP 26-27 Reapprops.pdf")
print("  - STOPS 26-27 Reapprops.pdf")
print("\n")

uploaded = files.upload()
print(f"\nUploaded {len(uploaded)} files: {list(uploaded.keys())}")

In [ ]:
def extract_reappropriation_chunks(pdf_path):
    """
    Extract reappropriation text chunks from a budget PDF.
    Each chunk starts with 'By chapter XX, section Y, of the laws of ZZZZ'
    and ends when 're. $XXXX' is found.
    """
    records = []
    
    with pdfplumber.open(pdf_path) as pdf:
        all_lines = []
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                all_lines.extend(text.splitlines())

    current_chunk = []
    capturing = False
    chapter_header = ""

    for line in all_lines:
        line = line.strip()

        # Detect "By chapter XX, section Y, of the laws of ZZZZ"
        chapter_match = re.search(r"(By\s+chapter\s+\d+,\s+section\s+\d+,?\s+of\s+the\s+laws\s+of\s+\d{4})", line, re.IGNORECASE)
        if chapter_match:
            chapter_header = chapter_match.group(1)
            current_chunk = [line]
            capturing = True
            continue

        if capturing:
            current_chunk.append(line)

            # Detect "re. $XXXX" (Reappropriation amount found)
            re_match = re.search(r"re\.\s?\$[\d,]+(?:\.\d{2})?", line)
            if re_match:
                if not current_chunk[0].startswith("By chapter"):
                    current_chunk.insert(0, chapter_header)

                records.append({"Reappropriation_Text": "\n".join(current_chunk)})
                current_chunk = [chapter_header]

    return records

print("Extraction function defined.")

In [ ]:
def parse_reappropriation_details(text):
    """
    Parse structured details from reappropriation text using regex.
    Extracts: Year, Chapter, Section, Appropriation ID, Original Amount, Reappropriation Amount
    """
    details = {
        "Year": None,
        "Chapter": None,
        "Section": None,
        "Appropriation_ID": None,
        "Original_Amount": None,
        "Reapprop_Amount": None
    }
    
    # Extract year from "laws of XXXX"
    year_match = re.search(r"laws\s+of\s+(\d{4})", text, re.IGNORECASE)
    if year_match:
        details["Year"] = year_match.group(1)
    
    # Extract chapter number
    chapter_match = re.search(r"chapter\s+(\d+)", text, re.IGNORECASE)
    if chapter_match:
        details["Chapter"] = chapter_match.group(1)
    
    # Extract section number
    section_match = re.search(r"section\s+(\d+)", text, re.IGNORECASE)
    if section_match:
        details["Section"] = section_match.group(1)
    
    # Extract appropriation ID - typically a 5-digit number in parentheses like (12345) or (302198C1)
    approp_id_match = re.search(r"\(([A-Z0-9]{5,10})\)", text)
    if approp_id_match:
        details["Appropriation_ID"] = approp_id_match.group(1)
    
    # Extract original appropriation amount - dollar amount before "re."
    # Pattern: number followed by dots then dollar amount
    orig_amount_match = re.search(r"\.{3,}\s*([\d,]+)\s*\.{3,}", text)
    if orig_amount_match:
        details["Original_Amount"] = orig_amount_match.group(1).replace(",", "")
    
    # Extract reappropriation amount - "re. $X,XXX" or "(re. $X,XXX)"
    reapprop_match = re.search(r"re\.\s?\$([\d,]+)", text)
    if reapprop_match:
        details["Reapprop_Amount"] = reapprop_match.group(1).replace(",", "")
    
    return details

print("Parsing function defined.")

In [ ]:
def process_pdf(pdf_path, budget_type, fiscal_year):
    """
    Process a single PDF and return a DataFrame with extracted details.
    """
    print(f"Processing: {pdf_path}")
    
    chunks = extract_reappropriation_chunks(pdf_path)
    print(f"  Found {len(chunks)} reappropriation entries")
    
    records = []
    for chunk in chunks:
        text = chunk["Reappropriation_Text"]
        details = parse_reappropriation_details(text)
        details["Budget_Type"] = budget_type
        details["Fiscal_Year"] = fiscal_year
        details["Full_Text"] = text
        records.append(details)
    
    return pd.DataFrame(records)

print("Processing function defined.")

In [ ]:
# Map ALL PDF files in directory (fixes issue with multiple upload batches)
import os

# Get all PDF files in current directory
all_pdf_files = [f for f in os.listdir('.') if f.lower().endswith('.pdf')]
print(f"Found {len(all_pdf_files)} PDF files: {all_pdf_files}")

file_mapping = {}

for filename in all_pdf_files:
    fname_lower = filename.lower()

    # Determine budget type
    if "atl" in fname_lower:
        budget_type = "ATL"
    elif "cap" in fname_lower:
        budget_type = "CAP"
    elif "stops" in fname_lower:
        budget_type = "STOPS"
    else:
        budget_type = "Unknown"

    # Determine fiscal year
    if "25-26" in fname_lower or "2526" in fname_lower:
        fiscal_year = "25-26"
    elif "26-27" in fname_lower or "2627" in fname_lower:
        fiscal_year = "26-27"
    else:
        fiscal_year = "Unknown"

    file_mapping[filename] = {"budget_type": budget_type, "fiscal_year": fiscal_year}
    print(f"{filename} -> {budget_type} ({fiscal_year})")

print(f"\nMapped {len(file_mapping)} files.")

In [ ]:
# Process all PDFs
all_data_25_26 = []
all_data_26_27 = []

for filename, info in file_mapping.items():
    df = process_pdf(filename, info["budget_type"], info["fiscal_year"])
    
    if info["fiscal_year"] == "25-26":
        all_data_25_26.append(df)
    else:
        all_data_26_27.append(df)

# Combine into single DataFrames
df_25_26 = pd.concat(all_data_25_26, ignore_index=True) if all_data_25_26 else pd.DataFrame()
df_26_27 = pd.concat(all_data_26_27, ignore_index=True) if all_data_26_27 else pd.DataFrame()

print(f"\n25-26 Total Records: {len(df_25_26)}")
print(f"26-27 Total Records: {len(df_26_27)}")

In [ ]:
# Preview 25-26 data
print("25-26 Reappropriations Sample:")
df_25_26[["Budget_Type", "Year", "Chapter", "Section", "Appropriation_ID", "Reapprop_Amount"]].head(10)

In [ ]:
# Preview 26-27 data
print("26-27 Reappropriations Sample:")
df_26_27[["Budget_Type", "Year", "Chapter", "Section", "Appropriation_ID", "Reapprop_Amount"]].head(10)

In [ ]:
def create_unique_key(row):
    """
    Create a unique identifier for each reappropriation based on:
    Budget Type + Year + Chapter + Section + Appropriation ID
    """
    parts = [
        str(row.get("Budget_Type", "")),
        str(row.get("Year", "")),
        str(row.get("Chapter", "")),
        str(row.get("Section", "")),
        str(row.get("Appropriation_ID", ""))
    ]
    return "|".join(parts)

# Add unique keys
df_25_26["Unique_Key"] = df_25_26.apply(create_unique_key, axis=1)
df_26_27["Unique_Key"] = df_26_27.apply(create_unique_key, axis=1)

print("Unique keys created.")
print(f"25-26 unique entries: {df_25_26['Unique_Key'].nunique()}")
print(f"26-27 unique entries: {df_26_27['Unique_Key'].nunique()}")

In [ ]:
# Find items in 25-26 that are NOT in 26-27 (DROPPED)
keys_25_26 = set(df_25_26["Unique_Key"].dropna())
keys_26_27 = set(df_26_27["Unique_Key"].dropna())

dropped_keys = keys_25_26 - keys_26_27
continued_keys = keys_25_26 & keys_26_27
new_keys = keys_26_27 - keys_25_26

print(f"DROPPED (in 25-26 but not in 26-27): {len(dropped_keys)}")
print(f"CONTINUED (in both years): {len(continued_keys)}")
print(f"NEW (in 26-27 but not in 25-26): {len(new_keys)}")

In [ ]:
# Create DataFrames for each category
df_dropped = df_25_26[df_25_26["Unique_Key"].isin(dropped_keys)].copy()
df_dropped["Status"] = "DROPPED"

df_continued_25 = df_25_26[df_25_26["Unique_Key"].isin(continued_keys)].copy()
df_continued_25["Status"] = "CONTINUED"

df_new = df_26_27[df_26_27["Unique_Key"].isin(new_keys)].copy()
df_new["Status"] = "NEW"

print(f"Dropped items: {len(df_dropped)}")
print(f"Continued items: {len(df_continued_25)}")
print(f"New items: {len(df_new)}")

In [ ]:
# Show dropped items by budget type
print("\n=== DROPPED REAPPROPRIATIONS BY BUDGET TYPE ===")
for bt in ["ATL", "CAP", "STOPS"]:
    dropped_bt = df_dropped[df_dropped["Budget_Type"] == bt]
    print(f"\n{bt}: {len(dropped_bt)} dropped items")
    if len(dropped_bt) > 0:
        print(dropped_bt[["Year", "Chapter", "Section", "Appropriation_ID", "Reapprop_Amount"]].head(10))

In [ ]:
# Calculate total dollars dropped by budget type
print("\n=== TOTAL DOLLARS DROPPED ===")
df_dropped["Reapprop_Amount_Num"] = pd.to_numeric(df_dropped["Reapprop_Amount"], errors="coerce")

for bt in ["ATL", "CAP", "STOPS"]:
    dropped_bt = df_dropped[df_dropped["Budget_Type"] == bt]
    total = dropped_bt["Reapprop_Amount_Num"].sum()
    print(f"{bt}: ${total:,.0f}")

print(f"\nGRAND TOTAL DROPPED: ${df_dropped['Reapprop_Amount_Num'].sum():,.0f}")

In [ ]:
# Export to Excel with multiple sheets
output_file = "Reappropriation_Comparison_25-26_vs_26-27.xlsx"

with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    # Summary sheet
    summary_data = {
        "Category": ["Total 25-26", "Total 26-27", "Dropped", "Continued", "New"],
        "Count": [len(df_25_26), len(df_26_27), len(df_dropped), len(df_continued_25), len(df_new)]
    }
    pd.DataFrame(summary_data).to_excel(writer, sheet_name="Summary", index=False)
    
    # Dropped items (most important)
    cols_to_export = ["Budget_Type", "Year", "Chapter", "Section", "Appropriation_ID", 
                     "Original_Amount", "Reapprop_Amount", "Full_Text"]
    df_dropped[cols_to_export].to_excel(writer, sheet_name="DROPPED", index=False)
    
    # Dropped by budget type
    for bt in ["ATL", "CAP", "STOPS"]:
        dropped_bt = df_dropped[df_dropped["Budget_Type"] == bt]
        if len(dropped_bt) > 0:
            dropped_bt[cols_to_export].to_excel(writer, sheet_name=f"Dropped_{bt}", index=False)
    
    # All 25-26 data
    df_25_26.to_excel(writer, sheet_name="All_25-26", index=False)
    
    # All 26-27 data
    df_26_27.to_excel(writer, sheet_name="All_26-27", index=False)

print(f"Excel file saved: {output_file}")

In [ ]:
# Download the Excel file
files.download(output_file)
print("Download started!")

## Results Summary

The Excel file contains:
- **Summary**: Overview of counts
- **DROPPED**: All reappropriations from 25-26 that are NOT in 26-27 (spending authority dropped)
- **Dropped_ATL/CAP/STOPS**: Dropped items by budget type
- **All_25-26**: Complete extraction from 25-26 PDFs
- **All_26-27**: Complete extraction from 26-27 PDFs